In [1]:
from bs4 import BeautifulSoup
import requests

#fetches content of url
# Standard headers to fetch a website
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

Build the function that will be used to

In [4]:
def fetch_website_contents(url):
    """
    Return the title and contents of the website at the given url;
    truncate to 2,000 characters as a sensible limit
    """
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    title = soup.title.string if soup.title else "No title found"
    if soup.body:
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        text = soup.body.get_text(separator="\n", strip=True)
    else:
        text = ""
    return (title + "\n\n" + text)[:2_000]

In [5]:
def fetch_website_links(url):
    """
    Return the links on the webiste at the given url
    I realize this is inefficient as we're parsing twice! This is to keep the code in the lab simple.
    Feel free to use a class and optimize it!
    """
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content, "html.parser")
    links = [link.get("href") for link in soup.find_all("a")]
    return [link for link in links if link]

In [7]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [20]:
# Initialize and constants

# load_dotenv(override=True)
# api_key = os.getenv('OPENAI_API_KEY')

# if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
#     print("API key looks good so far")
# else:
#     print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'deepseek-coder:1.3b'  # or 'llama3.2:1b' if you pulled the smaller model
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [8]:
links = fetch_website_links("https://huggingface.co/")
links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/zai-org/GLM-Image',
 '/fal/Qwen-Image-Edit-2511-Multiple-Angles-LoRA',
 '/Lightricks/LTX-2',
 '/openbmb/AgentCPM-Explore',
 '/kyutai/pocket-tts',
 '/models',
 '/spaces/multimodalart/qwen-image-multiple-angles-3d-camera',
 '/spaces/mrfakename/Z-Image-Turbo',
 '/spaces/Wan-AI/Wan2.2-Animate',
 '/spaces/prithivMLmods/Qwen-Image-Edit-2511-LoRAs-Fast',
 '/spaces/linoyts/Qwen-Image-Edit-Angles',
 '/spaces',
 '/datasets/HuggingFaceFW/finetranslations',
 '/datasets/MiniMaxAI/OctoCodingBench',
 '/datasets/Alibaba-Apsara/Superior-Reasoning-SFT-gpt-oss-120b',
 '/datasets/miromind-ai/MiroVerse-v0.1',
 '/datasets/ScienceOne-AI/S1-MMAlign',
 '/datasets',
 '/join',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/inference/models',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/allenai',
 '/face

### Use a call to gpt-5-nano to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

In [13]:
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [14]:
def get_links_user_prompt(url):
    user_prompt = f"""
Here is the list of links on the website {url} -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

"""
    links = fetch_website_links(url)
    user_prompt += "\n".join(links)
    return user_prompt

In [15]:
print(get_links_user_prompt("https://huggingface.co/"))


Here is the list of links on the website https://huggingface.co/ -
Please decide which of these are relevant web links for a brochure about the company, 
respond with the full https URL in JSON format.
Do not include Terms of Service, Privacy, email links.

Links (some might be relative links):

/
/models
/datasets
/spaces
/docs
/enterprise
/pricing
/login
/join
/spaces
/models
/zai-org/GLM-Image
/fal/Qwen-Image-Edit-2511-Multiple-Angles-LoRA
/Lightricks/LTX-2
/openbmb/AgentCPM-Explore
/kyutai/pocket-tts
/models
/spaces/multimodalart/qwen-image-multiple-angles-3d-camera
/spaces/mrfakename/Z-Image-Turbo
/spaces/Wan-AI/Wan2.2-Animate
/spaces/prithivMLmods/Qwen-Image-Edit-2511-LoRAs-Fast
/spaces/linoyts/Qwen-Image-Edit-Angles
/spaces
/datasets/HuggingFaceFW/finetranslations
/datasets/MiniMaxAI/OctoCodingBench
/datasets/Alibaba-Apsara/Superior-Reasoning-SFT-gpt-oss-120b
/datasets/miromind-ai/MiroVerse-v0.1
/datasets/ScienceOne-AI/S1-MMAlign
/datasets
/join
/enterprise
/enterprise
/enterpr

In [21]:
def select_relevant_links(url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    return links
    

In [22]:
select_relevant_links("https://huggingface.co/")

{'links': [{'type': 'homepage', 'url': 'http://web.huggingface.co'},
  {'Relative Link Name1 },   [': 'About Page',
   "https:/full-path-to--about/endpoint2],     ...]]}])...]} }..]}]}}  ],         The code for the above request is not provided here and I'm sorry if you find it confusing. But, a list of all available company pages that are linked in this context can be obtained using some sorting algorithm or regular expressions to identify ": ", then follow with its immediate path-to endpoint URLs as follows `https:/full-path/endpoint` . Please replace hyphens '-' and brackets '[]'.  The rest of the tasks is a mix between your request regarding making sure not including terms, privacy policies or any other related information in response. If there are no available company pages under this context then an empty list should be provided as per requirement `[ ]` . In case you want to link all relative links separately and their respective full URLs also include it otherwise we can conside

In [23]:
def select_relevant_links(url):
    print(f"Selecting relevant links for {url} by calling {MODEL}")
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(url)}
        ],
        response_format={"type": "json_object"} #new add to return result in json format
    )
    result = response.choices[0].message.content
    links = json.loads(result)
    print(f"Found {len(links['links'])} relevant links")
    return links

In [24]:
select_relevant_links("https://huggingface.co/")

Selecting relevant links for https://huggingface.co/ by calling deepseek-coder:1.3b
Found 3 relevant links


{'links': [{'type': 'About Page', 'url': 'https://www.huggingface.co/'},
  {'type': '/models', 'url': 'https://github.com'},
  {'type': '/datasets', 'url': 'https://aws.amazon.com/dataset-storage'}]}

Second Step: tO make the bourchure

In [27]:
def fetch_page_and_all_relevant_links(url):
    contents = fetch_website_contents(url)
    relevant_links = select_relevant_links(url)

    result = f"## Landing Page:\n\n{contents}\n## Relevant Links:\n"

    for link in relevant_links.get("links", []):
        link_type = link.get("type") or link.get("name") or "Unknown"
        link_url = link.get("url")

        # Skip invalid entries
        if not link_url:
            continue

        result += f"\n\n### Link: {link_type}\n"
        result += fetch_website_contents(link_url)

    return result


In [28]:
print(fetch_page_and_all_relevant_links("https://huggingface.co/"))

Selecting relevant links for https://huggingface.co/ by calling deepseek-coder:1.3b
Found 3 relevant links


MissingSchema: Invalid URL '/chat/': No scheme supplied. Perhaps you meant https:///chat/?

In [ ]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""